<a href="https://colab.research.google.com/github/drshahizan/Python_EDA/blob/main/assignment/ass5/hpdp/ANGKASA/Tool%201%20-%20TPOT/Assignment_5A_TPOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# **Assignment 5 - TPOTASSIGNMENT 4: FEATURE ENGINEERING**

Dataset Link: https://www.kaggle.com/datasets/ahmedshahriarsakib/usa-real-estate-dataset

## Group Members and Matriculation Numbers

1. **Member 1**
   - **Name:** Muhammad Ashraaf Bin Salleh
   - **Matriculation Number:** A21EC0068

2. **Member 2**
   - **Name:** Fauzan Aqil Bin Azman
   - **Matriculation Number:** A21EC0174

3. **Member 3**
   - **Name:** Muhammad Naquib Bin Zakaria
   - **Matriculation Number:** A20BE0161

4. **Member 4**
   - **Name:** Nik Amirul Ariff Bin Amran
   - **Matriculation Number:** A21EC0214



TPOT, which stands for "Tree-based Pipeline Optimization Tool," is a powerful Python library designed to automate the machine learning pipeline, making it an invaluable asset for data scientists and machine learning practitioners. Developed with the aim of simplifying the model selection and hyperparameter tuning process, TPOT leverages genetic programming to discover and optimize machine learning pipelines. Unlike traditional approaches that require manual intervention in the model selection and hyperparameter tuning stages, TPOT automates this complex process, enabling users to efficiently navigate through a vast search space of possible pipelines.

One of TPOT's notable features is its ability to explore diverse combinations of preprocessing steps, feature selection techniques, and machine learning models, adapting its search strategy to the unique characteristics of the dataset at hand. By employing genetic programming, TPOT evolves and refines its pipelines over multiple generations, gradually converging towards the most effective configuration for a given machine learning task. This automation not only accelerates the model development process but also helps users discover high-performing models that may have been overlooked through manual exploration.

Moreover, TPOT is designed to handle regression and classification tasks, allowing users to apply it to a wide range of predictive modeling scenarios. Its flexibility, ease of use, and automated optimization capabilities make TPOT a valuable tool for both beginners and experienced practitioners seeking efficient and effective solutions to complex machine learning challenges.

## Mounting google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install TPOT using the following command:

In [2]:
!pip install tpot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11938 sha256=c8c00524d235ee79693b5f781377398eda2e495335bcafdb8737d0d8b467b3d4
  Stored in directory: /root/.cache/pip/wheels/af/f9/87/bf5b3d565c2a007b4dae9d8142dccc85a9f164e517062dd519
Successfully built stopit


In [3]:
!pip install --upgrade scikit-learn
!apt-get install graphviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 21.5 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
graphviz is already the newest version (2.42.2-6).
0 upgraded, 0 newly installed, 0 to remove and 10 not upgraded.


## Importing any other necessary libraries

In [4]:
import pandas as pd
from tpot import TPOTClassifier
from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

## Read the file and Divide into chunk

In [5]:
file_path = '/content/drive/MyDrive/realtor-data.zip.csv'
chunk_size = 50000  # Adjust the chunk size based on your available RAM

# Create an empty DataFrame to store the chunks
chunks = []

for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    chunks.append(chunk)

# Concatenate chunks into a single DataFrame
data = pd.concat(chunks, axis=0, ignore_index=True)

## Data Information

In [6]:
data.head()

,status,bed,bath,acre_lot,city,state,zip_code,house_size,prev_sold_date,price
0,for_sale,3.0,2.0,0.12,Adjuntas,Puerto Rico,601.0,920.0,NaN,105000.0
1,for_sale,4.0,2.0,0.08,Adjuntas,Puerto Rico,601.0,1527.0,NaN,80000.0
2,for_sale,2.0,1.0,0.15,Juana Diaz,Puerto Rico,795.0,748.0,NaN,67000.0
3,for_sale,4.0,2.0,0.10,Ponce,Puerto Rico,731.0,1800.0,NaN,145000.0
4,for_sale,6.0,2.0,0.05,Mayaguez,Puerto Rico,680.0,NaN,NaN,65000.0


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 904966 entries, 0 to 904965
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   status          904966 non-null  object 
 1   bed             775126 non-null  float64
 2   bath            791082 non-null  float64
 3   acre_lot        638324 non-null  float64
 4   city            904894 non-null  object 
 5   state           904966 non-null  object 
 6   zip_code        904762 non-null  float64
 7   house_size      612080 non-null  float64
 8   prev_sold_date  445865 non-null  object 
 9   price           904895 non-null  float64
dtypes: float64(6), object(4)
memory usage: 69.0+ MB


## Data Cleaning and Pre-processing

In [8]:
columns_to_replace = ['bed', 'bath', 'house_size']
data[columns_to_replace] = data[columns_to_replace].fillna(0)

In [9]:
data = data.drop('prev_sold_date', axis=1)

In [10]:
data = data.drop('status', axis=1)

In [11]:
data = data.dropna(subset=['acre_lot', 'zip_code'])

In [12]:
data.head(80)

,bed,bath,acre_lot,city,state,zip_code,house_size,price
0,3.0,2.0,0.12,Adjuntas,Puerto Rico,601.0,920.0,105000.0
1,4.0,2.0,0.08,Adjuntas,Puerto Rico,601.0,1527.0,80000.0
2,2.0,1.0,0.15,Juana Diaz,Puerto Rico,795.0,748.0,67000.0
3,4.0,2.0,0.10,Ponce,Puerto Rico,731.0,1800.0,145000.0
4,6.0,2.0,0.05,Mayaguez,Puerto Rico,680.0,0.0,65000.0
...,...,...,...,...,...,...,...,...
79,0.0,0.0,0.37,Aguada,Puerto Rico,602.0,0.0,95000.0
80,0.0,0.0,0.23,Rincon,Puerto Rico,602.0,0.0,329000.0
81,0.0,0.0,0.68,Aguada,Puerto Rico,602.0,0.0,130000.0
82,0.0,0.0,0.25,Aguada,Puerto Rico,602.0,0.0,94770.0


# 1. Hyperparameter Tuning with TPOT:

## Splitting the Data into Features and Target

In [13]:
X = data.drop('price', axis=1)
y = data['price']

## Train-Test Split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Imputing null value into mean value

In [15]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')
X_train_numeric = X_train.select_dtypes(include=['number'])
X_train[X_train_numeric.columns] = imputer.fit_transform(X_train_numeric)

## Dropping unnecessary column that cannot be used

In [16]:
non_numeric_columns = X_train.select_dtypes(exclude=['number']).columns
X_train = X_train.drop(columns=non_numeric_columns)

## Running the TPOT

In [17]:
tpot = TPOTRegressor(verbosity=2, generations=5, population_size=20, random_state=42, n_jobs=-1)
tpot.fit(X_train, y_train)

Optimization Progress:   0%|          | 0/120 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -380643391240.4049

Generation 2 - Current best internal CV score: -136255062103.94263

Generation 3 - Current best internal CV score: -76898490101.93436

Generation 4 - Current best internal CV score: -76898490101.93436

Generation 5 - Current best internal CV score: -76898490101.93436

Best pipeline: KNeighborsRegressor(input_matrix, n_neighbors=71, p=2, weights=distance)


TPOTRegressor(generations=5, n_jobs=-1, population_size=20, random_state=42,
              verbosity=2)

## Evaluating the model

In [18]:
tpot.fitted_pipeline_.steps[-1][1].feature_importances_

AttributeError: ignored

In [ ]:
import matplotlib.pyplot as plt

feature_importances = tpot.fitted_pipeline_.steps[-1][1].feature_importances_
features = X_train.columns  # Assuming X_train is a DataFrame
plt.bar(features, feature_importances)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
tpot.fitted_pipeline_.graphviz()

# 2. Feature selection using **TPOT**

In [ ]:
tpot_fs = TPOTRegressor(verbosity=2, generations=5, population_size=20, random_state=42, n_jobs=-1,
                        config_dict='TPOT sparse', memory='auto', periodic_checkpoint_folder='/content/gdrive/MyDrive/tpot_checkpoint')

tpot_fs.fit(X_train, y_train)

## Getting the best pipeline

In [ ]:
best_model_fs = tpot_fs.fitted_pipeline_

## Applying feature selection on the dataset


In [ ]:
feature_selector = SelectFromModel(best_model_fs.steps[-1][1], prefit=True)
X_train_selected = feature_selector.transform(X_train)
X_test_selected = feature_selector.transform(X_test)

# 4. Conclusion


In conclusion, leveraging TPOT for automated machine learning proved to be a valuable approach in streamlining the feature engineering process. TPOT's genetic programming capabilities allowed for an exhaustive exploration of diverse combinations of preprocessing steps, feature selections, and regression models, ultimately leading to the discovery of an optimized machine learning pipeline.

Throughout the process, TPOT intelligently navigated the feature engineering landscape, adapting its search strategy to the unique characteristics of the dataset. The automation included handling missing values, encoding categorical variables, and selecting relevant features, all vital steps in preparing the data for regression modeling.

The decision to use TPOT for feature engineering was motivated by its ability to:

Automate Model Selection: TPOT's genetic programming systematically explored a variety of regression models, identifying the one that best suited the dataset. This eliminated the need for manual model selection, ensuring a comprehensive search across the model space.

Optimize Hyperparameters: TPOT not only chose the best model but also fine-tuned its hyperparameters. This automated hyperparameter optimization contributed to the overall performance improvement of the selected model.

Handle Complex Pipelines: TPOT's flexibility allowed it to consider complex machine learning pipelines, including diverse preprocessing steps. This adaptability was crucial in addressing the intricacies of the dataset without the need for manual intervention.

Enhance Reproducibility: The use of TPOT with specified random seeds ensured reproducibility, enabling consistent results across different runs and facilitating the tracking of the feature engineering and model selection process.

In summary, the application of TPOT for feature engineering provided a robust, automated solution that not only saved time and effort but also unearthed a high-performing machine learning pipeline. The decisions to automate feature engineering with TPOT were driven by the desire to efficiently explore the feature space, optimize model performance, and maintain a reproducible and adaptable workflow in the regression modeling task.